## Phân loại nội dung độc hại "Quora Insincere Questions Classification" - Kaggle




### **Cài đặt pytorch, kết nối google drive**

Cài đặt pytorch

In [1]:
pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████▌                  | 834.1MB 1.4MB/s eta 0:14:10tcmalloc: large alloc 1147494400 bytes == 0x5560d3c3c000 @  0x7fd66d342615 0x556099f14cdc 0x556099ff452a 0x556099f17afd 0x55609a008fed 0x556099f8b988 0x556099f864ae 0x556099f193ea 0x556099f8b7f0 0x556099f864ae 0x556099f193ea 0x556099f8832a 0x55609a009e36 0x556099f87853 0x55609a009e36 0x556099f87853 0x55609a009e36 0x556099f87853 0x55609a009e36 0x55609a08c3e1 0x556099fec6a9 0x556099f57cc4 0x556099f18559 0x556099f8c4f8 0x556099f1930a 0x556099f873b5 0x556099f867ad 0x556099f193ea 0x556099f873b5 0x556099f1930a 0x556099f873b5
     |█████████████████               | 1055.7MB 1.5MB/s eta 0:10:34tcmalloc: large alloc 1434370048 bytes == 0x556118292000 @  0x7fd66d342615 0x556099f14cdc 0x556099ff452a 0x556099f17afd 0x55609a008fed 0x556099f8b988 0x556099f864ae 0x556099f193ea 0x556099f8b7f0 0x556099f864ae 0x556099f193ea 0x556099f8832a 0x55609a009e36 0x556099f87853 0

Kết nối drive

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Import thư viện sử dụng

In [6]:
# standard imports
import time
import random
import os
from IPython.display import display
import numpy as np
import pandas as pd

# pytorch imports
import torch
import torch.nn as nn
import torch.utils.data

# imports for preprocessing the questions
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# cross validation and metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

# progress bars
from tqdm import tqdm
tqdm.pandas()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


### **Hiển thị, tiền xử lý dữ liệu**

Load data

In [7]:
train_df = pd.read_csv("/content/gdrive/MyDrive/LSTM_attention/train.csv")
test_df = pd.read_csv("/content/gdrive/MyDrive/LSTM_attention/test.csv")
print('Train data dimension: ', train_df.shape)
display(train_df.head())
print('Test data dimension: ', test_df.shape)
display(test_df.head())

Train data dimension:  (1306122, 3)


,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


Test data dimension:  (375806, 2)


,qid,question_text
0,0000163e3ea7c7a74cd7,Why do so many women become so rude and arroga...
1,00002bd4fb5d505b9161,When should I apply for RV college of engineer...
2,00007756b4a147d2b0b3,What is it really like to be a nurse practitio...
3,000086e4b7e1c7146103,Who are entrepreneurs?
4,0000c4c3fbe8785a3090,Is education really making good people nowadays?


Ngẫu nhiên bằng pytorch

In [8]:
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything()

Xây dựng hàm tìm kiếm thershold_search

In [9]:
def threshold_search(y_true, y_proba):
    best_threshold = 0
    best_score = 0
    for threshold in tqdm([i * 0.01 for i in range(100)]):
        score = f1_score(y_true=y_true, y_pred=y_proba > threshold)
        if score > best_score:
            best_threshold = threshold
            best_score = score
    search_result = {'threshold': best_threshold, 'f1': best_score}
    return search_result

Hàm sigmoid

In [10]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

### Tiền xử lý dữ liệu

Cài đặt các tham số:
- embed_size = 300: số chiều vectơ nhúng cho mỗi từ
- max_features = 95000: số lượng các từ sử dụng
- max_len = 70: số lượng từ lớn nhất trong câu hỏi người dùng

In [11]:
embed_size = 300 # how big is each word vector
max_features = 95000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 70 # max number of words in a question to use

Làm sạch văn bản, loại bỏ các ký tự thừa, dấu câu,...

In [12]:
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]
def clean_text(x):
    x = str(x)
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
    return x

Chuyển dữ liệu về chữ thường, lấp đầy các giá trị thiếu và tokenize các câu

In [13]:
train_df["question_text"] = train_df["question_text"].str.lower()
test_df["question_text"] = test_df["question_text"].str.lower()

train_df["question_text"] = train_df["question_text"].apply(lambda x: clean_text(x))
test_df["question_text"] = test_df["question_text"].apply(lambda x: clean_text(x))

# fill up the missing values
x_train = train_df["question_text"].fillna("_##_").values
x_test = test_df["question_text"].fillna("_##_").values

# Tokenize the sentences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(x_train))
x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)

# Pad the sentences 
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

# Get the target values
y_train = train_df['target'].values

In [14]:
print(train_df["question_text"][0])
print(x_train[0])
print(x_train.shape)
print(x_test.shape)

how did quebec nationalists see their province as a nation in the 1960s ? 
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    9   51 6611 7095  162   58 6061   38    4 1147    6    1 8322]
(1306122, 70)
(375806, 70)


- Mã hoá 1306122 mỗi câu hỏi trong x_train thành mỗi câu hỏi có độ dài 70 chứa các chỉ số trong từ điển (đã padding dữ liệu).
-  Mã hoá 375806 mỗi câu hỏi trong x_test thành vector có độ dài 70.

### Khởi tạo ma trận nhúng Embeddings Matrix

In [15]:
def load_glove(word_index):
    EMBEDDING_FILE = '/content/gdrive/MyDrive/LSTM_attention/glove.840B.300d.txt'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')[:300]
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))
    
    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = -0.005838499,0.48782197
    embed_size = all_embs.shape[1]

    # word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
            
    return embedding_matrix 
    
def load_fasttext(word_index):    
    EMBEDDING_FILE = '/content/gdrive/MyDrive/LSTM_attention/wiki-news-300d-1M.vec'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE) if len(o)>100)

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]

    # word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector

    return embedding_matrix

def load_para(word_index):
    EMBEDDING_FILE = '/content/gdrive/MyDrive/LSTM_attention/paragram_300_sl999.txt'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, encoding="utf8", errors='ignore') if len(o)>100)

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = -0.0053247833,0.49346462
    embed_size = all_embs.shape[1]

    # word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
    
    return embedding_matrix

In [16]:
seed_everything()

glove_embeddings = load_glove(tokenizer.word_index)
paragram_embeddings = load_para(tokenizer.word_index)

embedding_matrix = np.mean([glove_embeddings, paragram_embeddings], axis=0)
np.shape(embedding_matrix)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  exec(code_obj, self.user_global_ns, self.user_ns)


(95000, 300)

### **Định nghĩa mô hình**

Kỹ thuật k-fold cross-validation:
- Xáo trộn dữ liệu huấn luyện một cách ngẫu nhiên.
- Chia tập huấn luyện k=5 nhóm.
- Với mỗi một nhóm: sử dụng 4 nhóm còn lại để huấn luyện mô hình và sử dụng nhóm đó đánh giá hiệu quả mô hình.


In [17]:
splits = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=10).split(x_train, y_train))

In [18]:
splits

[(array([      0,       1,       2, ..., 1306119, 1306120, 1306121]),
  array([      9,      10,      12, ..., 1306095, 1306109, 1306112])),
 (array([      3,       4,       5, ..., 1306119, 1306120, 1306121]),
  array([      0,       1,       2, ..., 1306099, 1306100, 1306104])),
 (array([      0,       1,       2, ..., 1306118, 1306119, 1306121]),
  array([      4,       5,       7, ..., 1306108, 1306114, 1306120])),
 (array([      0,       1,       2, ..., 1306119, 1306120, 1306121]),
  array([      6,      13,      21, ..., 1306113, 1306116, 1306118])),
 (array([      0,       1,       2, ..., 1306116, 1306118, 1306120]),
  array([      3,      15,      17, ..., 1306117, 1306119, 1306121]))]

Cơ chế Attention xây dựng mô hình

In [19]:
class Attention(nn.Module):
    def __init__(self, feature_dim, step_dim, bias=True, **kwargs):
        super(Attention, self).__init__(**kwargs)
        
        self.supports_masking = True

        self.bias = bias
        self.feature_dim = feature_dim
        self.step_dim = step_dim
        self.features_dim = 0
        
        weight = torch.zeros(feature_dim, 1)
        nn.init.xavier_uniform_(weight)
        self.weight = nn.Parameter(weight)
        
        if bias:
            self.b = nn.Parameter(torch.zeros(step_dim))
        
    def forward(self, x, mask=None):
        feature_dim = self.feature_dim
        step_dim = self.step_dim

        eij = torch.mm(
            x.contiguous().view(-1, feature_dim), 
            self.weight
        ).view(-1, step_dim)
        
        if self.bias:
            eij = eij + self.b
            
        eij = torch.tanh(eij)
        a = torch.exp(eij)
        
        if mask is not None:
            a = a * mask

        a = a / torch.sum(a, 1, keepdim=True) + 1e-10

        weighted_input = x * torch.unsqueeze(a, -1)
        return torch.sum(weighted_input, 1)
    
class SpatialDropout(nn.Dropout2d):
    def forward(self, x):
        x = x.unsqueeze(2)    # (N, T, 1, K)
        x = x.permute(0, 3, 2, 1)  # (N, K, 1, T)
        x = super(SpatialDropout, self).forward(x)  # (N, K, 1, T), some features are masked
        x = x.permute(0, 3, 2, 1)  # (N, T, 1, K)
        x = x.squeeze(2)  # (N, T, K)
        return x

Xây dựng mạng Neural Network cho mô hình

In [20]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        
        hidden_size = 40
        
        self.embedding = nn.Embedding(max_features, embed_size)
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = False
        
        self.embedding_dropout = SpatialDropout(0.1)
        self.lstm = nn.LSTM(embed_size, hidden_size, bidirectional=True, batch_first=True)
        self.gru = nn.GRU(hidden_size * 2, hidden_size, bidirectional=True, batch_first=True)
        
        self.lstm_attention = Attention(hidden_size * 2, maxlen)
        self.gru_attention = Attention(hidden_size * 2, maxlen)
        
        self.linear = nn.Linear(320, 16)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.1)
        self.out = nn.Linear(16, 1)
    
    def forward(self, x):
        h_embedding = self.embedding(x)
        h_embedding = self.embedding_dropout(h_embedding)
        
        h_lstm, _ = self.lstm(h_embedding)
        h_gru, _ = self.gru(h_lstm)
        
        h_lstm_atten = self.lstm_attention(h_lstm)
        h_gru_atten = self.gru_attention(h_gru)
        
        # global average pooling
        avg_pool = torch.mean(h_gru, 1)
        # global max pooling
        max_pool, _ = torch.max(h_gru, 1)
        
        conc = torch.cat((h_lstm_atten, h_gru_atten, avg_pool, max_pool), 1)
        conc = self.relu(self.linear(conc))
        conc = self.dropout(conc)
        out = self.out(conc)
        
        return out

### **Qúa trình huấn luyện**

In [21]:
batch_size = 512 # how many samples to process at once
n_epochs = 6 # how many times to iterate over all samples

In [22]:
# matrix for the out-of-fold predictions
train_preds = np.zeros((len(train_df)))
# matrix for the predictions on the test set
test_preds = np.zeros((len(test_df)))

# always call this before training for deterministic results
seed_everything()

x_test_cuda = torch.tensor(x_test, dtype=torch.long).cuda()
test = torch.utils.data.TensorDataset(x_test_cuda)
test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False)

for i, (train_idx, valid_idx) in enumerate(splits):    
    # split data in train / validation according to the KFold indeces
    # also, convert them to a torch tensor and store them on the GPU (done with .cuda())
    x_train_fold = torch.tensor(x_train[train_idx], dtype=torch.long).cuda()
    y_train_fold = torch.tensor(y_train[train_idx, np.newaxis], dtype=torch.float32).cuda()
    x_val_fold = torch.tensor(x_train[valid_idx], dtype=torch.long).cuda()
    y_val_fold = torch.tensor(y_train[valid_idx, np.newaxis], dtype=torch.float32).cuda()
    
    model = NeuralNet()
    # make sure everything in the model is running on the GPU
    model.cuda()

    # define binary cross entropy loss
    # note that the model returns logit to take advantage of the log-sum-exp trick 
    # for numerical stability in the loss
    loss_fn = torch.nn.BCEWithLogitsLoss(reduction='mean')
    optimizer = torch.optim.Adam(model.parameters())

    train = torch.utils.data.TensorDataset(x_train_fold, y_train_fold)
    valid = torch.utils.data.TensorDataset(x_val_fold, y_val_fold)
    
    train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
    valid_loader = torch.utils.data.DataLoader(valid, batch_size=batch_size, shuffle=False)
    
    print(f'Fold {i + 1}')
    
    for epoch in range(n_epochs):
        # set train mode of the model. This enables operations which are only applied during training like dropout
        start_time = time.time()
        model.train()
        avg_loss = 0.  
        for x_batch, y_batch in tqdm(train_loader, disable=True):
            # Forward pass: compute predicted y by passing x to the model.
            y_pred = model(x_batch)

            # Compute and print loss.
            loss = loss_fn(y_pred, y_batch)

            # Before the backward pass, use the optimizer object to zero all of the
            # gradients for the Tensors it will update (which are the learnable weights
            # of the model)
            optimizer.zero_grad()

            # Backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()

            # Calling the step function on an Optimizer makes an update to its parameters
            optimizer.step()
            avg_loss += loss.item() / len(train_loader)
            
        # set evaluation mode of the model. This disabled operations which are only applied during training like dropout
        model.eval()
        
        # predict all the samples in y_val_fold batch per batch
        valid_preds_fold = np.zeros((x_val_fold.size(0)))
        test_preds_fold = np.zeros((len(test_df)))
        
        avg_val_loss = 0.
        for i, (x_batch, y_batch) in enumerate(valid_loader):
            y_pred = model(x_batch).detach()
            
            avg_val_loss += loss_fn(y_pred, y_batch).item() / len(valid_loader)
            valid_preds_fold[i * batch_size:(i+1) * batch_size] = sigmoid(y_pred.cpu().numpy())[:, 0]
        
        elapsed_time = time.time() - start_time 
        print('Epoch {}/{} \t loss={:.4f} \t val_loss={:.4f} \t time={:.2f}s'.format(
            epoch + 1, n_epochs, avg_loss, avg_val_loss, elapsed_time))
        
    # predict all samples in the test set batch per batch
    for i, (x_batch,) in enumerate(test_loader):
        y_pred = model(x_batch).detach()

        test_preds_fold[i * batch_size:(i+1) * batch_size] = sigmoid(y_pred.cpu().numpy())[:, 0]

    train_preds[valid_idx] = valid_preds_fold
    test_preds += test_preds_fold / len(splits)

Fold 1
Epoch 1/6 	 loss=0.1258 	 val_loss=0.1081 	 time=61.63s
Epoch 2/6 	 loss=0.1077 	 val_loss=0.1034 	 time=62.18s
Epoch 3/6 	 loss=0.1029 	 val_loss=0.1006 	 time=62.02s
Epoch 4/6 	 loss=0.0994 	 val_loss=0.1015 	 time=62.01s
Epoch 5/6 	 loss=0.0964 	 val_loss=0.1000 	 time=62.39s
Epoch 6/6 	 loss=0.0940 	 val_loss=0.0985 	 time=62.75s
Fold 2
Epoch 1/6 	 loss=0.1285 	 val_loss=0.1086 	 time=62.70s
Epoch 2/6 	 loss=0.1088 	 val_loss=0.1040 	 time=61.74s
Epoch 3/6 	 loss=0.1038 	 val_loss=0.1013 	 time=62.43s
Epoch 4/6 	 loss=0.1003 	 val_loss=0.1002 	 time=62.55s
Epoch 5/6 	 loss=0.0972 	 val_loss=0.0995 	 time=62.31s
Epoch 6/6 	 loss=0.0946 	 val_loss=0.0986 	 time=62.40s
Fold 3
Epoch 1/6 	 loss=0.1287 	 val_loss=0.1104 	 time=62.43s
Epoch 2/6 	 loss=0.1087 	 val_loss=0.1032 	 time=62.38s
Epoch 3/6 	 loss=0.1034 	 val_loss=0.1006 	 time=62.70s
Epoch 4/6 	 loss=0.0999 	 val_loss=0.0993 	 time=62.28s
Epoch 5/6 	 loss=0.0973 	 val_loss=0.0991 	 time=62.34s
Epoch 6/6 	 loss=0.0949 	 v

In [23]:
search_result = threshold_search(y_train, train_preds)
search_result

100%|██████████| 100/100 [00:38<00:00,  2.60it/s]


{'f1': 0.6823369359836746, 'threshold': 0.39}